<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pickle
with open('extracted_data/pos_data.pkl', 'br') as f:
    pos_data = pickle.load(f)

In [13]:
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
import torch

pdb = pos_data['6B3J']
coo = pdb['coo']

k_neigh = 3
threshold = 20000
neigh = NearestNeighbors(k_neigh)
neigh.fit(coo)
neigh_idx = neigh.kneighbors(coo, return_distance=False)[:, 1:]


In [67]:
len('GSHMKEPKLSPKQERFIEEYFINDMNATKAAIAAGYSKNSASAIGAENLQKPAIRARIDARLK')

63

In [7]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px

In [54]:
with open('map/at-sym-to-idx.pkl', 'br') as f:
    at_sym_to_idx = pickle.load(f)
at_sym_to_idx = {v:k for k, v in at_sym_to_idx.items()}

df = pd.DataFrame(coo, columns=['x', 'y', 'z'])
df['atom'] = [at_sym_to_idx[i] for i in pdb['at_seq']]
df.head()

,x,y,z,atom
0,98.1250,152.50,95.6875,N
1,97.5625,152.50,97.0000,C
2,98.5000,151.75,97.9375,C
3,98.8125,150.50,97.7500,O
4,96.1875,151.75,97.0000,C


In [62]:
fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='atom', opacity=0.7)
fig.update_traces(marker=dict(size=3),
                  selector=dict(mode='markers'))
for i, j in neigh_idx:
    x = [coo[i, 0], coo[j, 0]]
    y = [coo[i, 1], coo[j, 1]]
    z = [coo[i, 2], coo[j, 2]]
    fig.add_trace(
        go.Scatter3d(
        x=x, y=y, z=z, mode='lines', showlegend=False,
        line=dict(
            color='darkblue',
            width=2
        )
))
fig.write_html("6B3J.html")


In [6]:
fig = go.Figure(data=go.Scatter3d(
    x=coo[:, 0], y=coo[:, 1], z=coo[:, 2],
    marker=dict(
        size=2,
        color=pdb['at_seq']
    )
))

fig.show()